# Development Scratch

This notebook contains ideas/scratch/notes for development

In [1]:
import os
import sys
from distutils.util import strtobool
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ['DJANGO_SETTINGS_MODULE'] = 'schedule_bot.settings'

django.setup()

In [2]:
import datetime
import django
import pandas as pd

from availabilities.models import *
from availabilities.utils import check_availabilities
from availabilities.utils import get_availabilities_for_participants
from events.models import *
from events.utils import get_all_event_participants
from collections import defaultdict
from events.utils import string_to_date_time, create_event

In [3]:
FEB_3_9AM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_3_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_4_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_4_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_5_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_5_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [4]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_3_9AM,
    "date_time_end": FEB_3_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_4_2PM,
    "date_time_end": FEB_4_5PM,
    "is_available": False
}

andrew, __ = User.objects.get_or_create(username="Andrew")
jessie, __ = User.objects.get_or_create(username="Jessie")
luna, __ = User.objects.get_or_create(username="Luna")

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)


# Generates proposed times
proposed_times = [
    (FEB_3_2PM, FEB_3_5PM),
    (FEB_4_2PM, FEB_4_5PM),
    (FEB_5_2PM, FEB_5_5PM),
]

# Generates event
create_event(
    host_user=andrew,
    event_name="Hockey Party",
    invitees=[jessie, luna],
    proposed_times=proposed_times
)

event = Event.objects.get(
    name="Hockey Party",
)


# Peeks
event_time_df = pd.DataFrame(EventTime.objects.filter(event=event).values())
event_time_df

,id,event_id,date_time_start,date_time_end
0,16,4,2022-02-03 14:00:00+00:00,2022-02-03 17:00:00+00:00
1,17,4,2022-02-04 14:00:00+00:00,2022-02-04 17:00:00+00:00
2,18,4,2022-02-05 14:00:00+00:00,2022-02-05 17:00:00+00:00


In [5]:
from events.finite_state_machine import UserEventMachine

# Reset
uets = UserEventTime.objects.filter(
    user=jessie,
    event_time__event=event
)
for uet in uets:
    uet.is_active=False
    uet.has_seen=False
    uet.explicit_response=UserEventTime.NO_RESPONSE
    uet.save()

user_event = UserEvent.objects.get(user=jessie, event=event)
user_event.state = UserEvent.NO_COMMUNICATION
user_event.save()
SuggestedDate.objects.all().delete()
EventTime.objects.filter(date_time_start=string_to_date_time("2022-2-17 4pm")).delete()

user_event = UserEvent.objects.get(user=jessie, event=event)
user_event_machine = UserEventMachine(user_event)

user_event_machine.send_initial_text()
user_event_machine.process_input_text("No")
user_event_machine.process_suggestion_text("2022-2-17 4pm")
user_event_machine.process_validation_text("Yes")

Jessie, would 2022-02-05 14:00:00+00:00 to 2022-02-05 17:00:00+00:00 work for you?
Jessie's new state: waiting_response
There are no viable event times, can you please suggest a time that works for you?
Jessie's new state: waiting_suggestion
Did you mean: 2022-02-17 16:00:00 - 2022-02-17 19:00:00?
Jessie's new state: waiting_validation
Andrew, would 2022-02-17 16:00:00+00:00 to 2022-02-17 19:00:00+00:00 work for you?
Andrew's new state: waiting_response
Luna, would 2022-02-17 16:00:00+00:00 to 2022-02-17 19:00:00+00:00 work for you?
Luna's new state: waiting_response
Jessie's new state: waiting_for_others


/Users/andrewlim/projects/event_scheduler/venv-event-scheduler/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField EventTime.date_time_start received a naive datetime (2022-02-17 16:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/andrewlim/projects/event_scheduler/venv-event-scheduler/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField SuggestedDate.interpreted_start received a naive datetime (2022-02-17 16:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/Users/andrewlim/projects/event_scheduler/venv-event-scheduler/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1416: RuntimeWarning: DateTimeField SuggestedDate.interpreted_end received a naive datetime (2022-02-17 19:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s recei

In [10]:
user_event = UserEvent.objects.get(user=andrew, event=event)
user_event_machine = UserEventMachine(user_event)
user_event_machine.process_input_text("No")

There are no viable event times, can you please suggest a time that works for you?
Andrew's new state: waiting_suggestion


In [4]:
pd.DataFrame(UserEvent.objects.values())

,id,state,user_id,event_id,is_required,is_host
0,10,waiting_suggestion,1,4,True,True
1,11,waiting_for_others,2,4,True,False
2,12,waiting_for_others,3,4,True,False


In [ ]:
# Event.objects.all().delete()